#### Basic stuff before all

In [1]:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="wapo"
)

mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)

#query = "SELECT * FROM reports WHERE id IN "+buf
#df_questions = pd.read_sql(query, con=mydb)



(u'reports',)


In [2]:
pd.set_option('display.max_colwidth', -1)

In [3]:
questions = pd.read_csv("questions.csv")
answers = pd.read_csv("answers.csv")
#questions

In [4]:
resultFile ="~/git/WapoDataMining/results/mlt_allfilter_content40_title10.txt"
def heavyInspect(topicId,resultFile,questions_df,answers_df):
    # read the result file
    myresult_df = pd.read_csv(resultFile, header=None, delim_whitespace=True)
    
    # extract first 5 cols of given topic id
    myresult_df = myresult_df.loc[myresult_df[0]==topicId][0:7]
    
    my_top_5_id = myresult_df[2]
    #print(my_top_5_id[0])
    
    # construct sql query and get result as a df for the ids
    buf = "("
    l = 5
    for i in range(5):
        buf+="'"+my_top_5_id[i]+"'"
        l = l-1
        if(l!=0):
            buf+=","

    buf+=")"
    #print(buf)
    
    query = "SELECT * FROM reports WHERE id IN "+buf
    myresult_df = pd.read_sql(query, con=mydb)
    question_df = questions.loc[questions["topic_no"]==topicId]
    answers_df = answers.loc[(answers["score"]>=16) & (answers["topic_no"]==topicId)]
    
    return myresult_df,question_df,answers_df
    


In [5]:
myresult_df,question_df,actual_result_df = heavyInspect(321,resultFile,questions,answers)


In [6]:
cols = ["title","content"]

question_df[cols]

title  \
28  Despite a big year for women in politics, national legislatures are still dominated by men   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [7]:
myresult_df[cols]

title  \
0  At Davos, Sheryl Sandberg’s silence on politics and quotas says it all                              
1  Despite a big year for women in politics, national legislatures are still dominated by men          
2  The changing face of women’s political participation in the Middle East                             
3  So the U.K. has its second female prime minister. What policies put more women in power?            
4  Twenty years after the most important U.N. conference on women, what – if anything – has changed?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [8]:
actual_result_df[cols]

title  \
40    Time to celebrate women’s gains in the U.K. election? Not so fast.                         
121   Where do African women have more power? Surprise — in countries emerging from war.         
276   The U.S. is catching up to Europe on gender equality                                       
528   A reality check about U.S. women in power                                                  
1073  Women could soon lead the three largest Western economies                                  
3881  Does Western pressure for gender equality help?                                            
3956  Egyptian women feel excluded, despite the promise of the revolution                        
4870  Why quotas are needed to achieve gender equality                                           
4892  So the U.K. has its second female prime minister. What policies put more women in power?   
6078  Ecuador votes on Sunday. These are 5 things you need to know.                              
6473  Why quotas are needed to achieve gender equality                                           
6478  What influence do women wield in African governments?                                      
7047  Rwanda is beating the United States in gender equality                                     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            